In [0]:
%pip install pytest

In [0]:
dbutils.library.restartPython()

In [0]:
# pytest.main runs our tests directly in the notebook environment, providing
# fidelity for Spark and other configuration variables.
#
# A limitation of this approach is that changes to the test will be
# cache by Python's import caching mechanism.
#
# To iterate on tests during development, we restart the Python process 
# and thus clear the import cache to pick up changes.


import pytest
import os
import sys
from pathlib import Path
import pprint

# Run all tests in the dab_demo.
notebook_path = Path(dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get())
repo_root = notebook_path.parent.parent
repo_root = Path(f'/Workspace/{repo_root}')
print('repo_root', repo_root)
os.chdir(repo_root)
%pwd

In [0]:


sys.path.insert(0, str(repo_root / 'src'))
sys.path.insert(0, str(repo_root))

# # 충돌 가능성이 있는 경로들 정의
tests_path = str(repo_root / 'tests')
# suspicious_parent_path = str(repo_root.parent) # <-- 문제의 상위 경로

# 충돌 경로 제거
if tests_path in sys.path:
    print(f"Removing tests path: {tests_path}")
    sys.path.remove(tests_path)

# if suspicious_parent_path in sys.path:
#     print(f"Removing suspicious parent path: {suspicious_parent_path}")
#     sys.path.remove(suspicious_parent_path)

print("\nCleaned sys.path:")
pprint.pprint(sys.path)

# Skip writing pyc files on a readonly filesystem.
sys.dont_write_bytecode = True

# 1. 위젯 값 가져오기
test_file = dbutils.widgets.get('test_file')
try:
    from databricks.connect import DatabricksSession

    pytest_args = [
    "-p", "no:cacheprovider",
    f"--debug=/tmp/pytestdebug_{os.getpid()}.log",
    "-v",
    "--tb=short",
    "-c", "/dev/null"  # <-- 이 옵션을 추가하여 모든 ini 파일을 무시

    ]
except ImportError:
    pytest_args = [
    "-p", "no:cacheprovider",
    # --debug 대신 --debug-log로 파일 위치를 명시적으로 지정
    f"--debug=/tmp/pytestdebug_{os.getpid()}.log",
    "-v",
    "--tb=short"
    ]


test_file = dbutils.widgets.get('test_file')
if test_file:
    target_path = repo_root / 'tests' / test_file
    if not target_path.is_file():
        raise FileNotFoundError(f"Test file not found at '{target_path}'")
    pytest_args.insert(0, str(target_path))
else:
    pytest_args.insert(0, "tests/")

print(f"Running pytest with arguments: {pytest_args}")

retcode = pytest.main(pytest_args)
print(retcode)
# assert retcode == 0, 'The pytest invocation failed. See the log above for details.'